In [1]:
import numpy as np
import pandas as pd
import math
import operator
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
import csv
import random
import warnings
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

In [2]:
df=pd.read_csv('./data/iris.data.txt')
df.columns= [" sepal_length","sepal_width","petal_length","petal_width","species"]
df.head(5)
df.shape

(149, 5)

In [3]:
df.to_csv('iris.data.csv')

### Handle data

In [4]:
import csv

with open('./data/iris.data.txt', 'r') as csvfile:

    lines = csv.reader(csvfile)

    for row in lines :

        print (', '.join(row))

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [42]:

def loadDataset(filename, split):
    trainingSet=[]
    trainingTarget=[]
    testSet=[]
    testTarget=[]
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)

        for x in range(len(dataset)-1):

            for y in range(4):

                dataset[x][y] = float(dataset[x][y])
                        
            if random.random() < split:

                trainingSet.append(dataset[x][0:len(dataset[0])-1])
                trainingTarget.append(dataset[x][4])

            else:

                testSet.append(dataset[x][0:len(dataset[0])-1])
                testTarget.append(dataset[x][4])
                
    return np.array(trainingSet),np.array(trainingTarget),np.array(testSet),np.array(testTarget)



In [43]:
x_train,y_train,x_test,y_test=loadDataset('./data/iris.data.txt', 0.66)

In [44]:
print(x_train.dtype)
print(x_test.dtype)

float64
float64


In [45]:
print(x_train.shape)
print(x_test.shape)

(90, 4)
(59, 4)


### Similarity

In [142]:
def euclideanDistance(instance1, instance2):
    
    d = 0
    length=len(instance2)
    for i in range(length):
        d += pow((instance1[i] - instance2[i]), 2)
    return math.sqrt(d)

In [143]:
data1 = [2, 2, 2]

data2 = [4, 4, 4]

distance = euclideanDistance(data1, data2)

print(distance)


3.4641016151377544


### Neighbors

In [187]:
def getNeighbors(trainingSet,trainingTarget, testInstance, k):

    distances = []

    length = len(testInstance)

    for x in range(len(trainingSet)):

        dist = euclideanDistance(testInstance, trainingSet[x])

        distances.append((trainingSet[x],trainingTarget[x], dist))

        distances.sort(key=operator.itemgetter(2))

    neighbors = []

    for x in range(k):

        neighbors.append((distances[x][0],distances[x][1]))

    return neighbors




In [188]:
trainSet = [[2, 2, 2], [4, 4, 4],[2, 2, 2], [4, 4, 4],[2, 2, 2], [4, 4, 4],[2, 2, 2], [4, 4, 4]]
traint=['a','b','a','b','a','b','a','b']
testInstance = [5, 5, 5]

k = 3

neighbors = getNeighbors(trainSet,traint, testInstance, k)

print(neighbors)

[([4, 4, 4], 'b'), ([4, 4, 4], 'b'), ([4, 4, 4], 'b')]


###  Response

In [189]:

def getResponse(neighbors):

    classVotes = {}

    for x in range(len(neighbors)):

        response = neighbors[x][-1] #complete with appropriate number

        if response in classVotes:

            classVotes[response] += 1
        else:
             classVotes[response] = 1

    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    

    return sortedVotes[0][0]

   

In [190]:
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

response = getResponse(neighbors)

print(response)

a


### Accuracy

In [191]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))


In [192]:
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
test = ['a', 'v', 'a']

predictions = ['a', 'a', 'a']

accuracy = getAccuracy(test, predictions)

print(accuracy)

0.6666666666666666


### Main

In [197]:
def main():
    
    split = 0.66
    x_train,y_train,x_test,y_test=loadDataset('./data/iris.data.txt', split)
    predictions=[]
    best_acc = 0
    best_k = 1
    

    for i in range(1,10):
        

        for x in range(len(x_test)):
            
            neighbors = getNeighbors(x_train,y_train, x_test[x], i)
            res = getResponse(neighbors)
            predictions.append(res)
        accuracy = getAccuracy(y_test, predictions)
        if(best_acc < accuracy):
            best_acc = accuracy
            best_k = i
            
    print('Accuracy: ', accuracy)
    print('best k :', best_k)

main()


Accuracy:  0.9821428571428571
best k : 1


## test with another distance metric

### 

In [186]:
from numpy.linalg import norm
# cosine similarity 
def cosine_sim(instance1, instance2):
    cos_sim = 0
    length=len(instance2)
    for x in range(length):
        cos_sim = np.dot(instance1[x], instance2[x])/(norm(instance1[x])*norm(instance2[x]))
    return cos_sim

In [200]:
def getNeighbors1(trainingSet,trainingTarget, testInstance, k):

    distances = []

    length = len(testInstance)

    for x in range(len(trainingSet)):

        dist = cosine_sim(testInstance, trainingSet[x])

        distances.append((trainingSet[x],trainingTarget[x], dist))

        distances.sort(key=operator.itemgetter(2))

    neighbors = []

    for x in range(k):

        neighbors.append((distances[x][0],distances[x][1]))

    return neighbors


In [201]:
def main():
    
    split = 0.66
    x_train,y_train,x_test,y_test=loadDataset('./data/iris.data.txt', split)
    predictions=[]
    best_acc = 0
    best_k = 1
    

    for i in range(1,10):
        

        for x in range(len(x_test)):
            
            neighbors = getNeighbors1(x_train,y_train, x_test[x], i)
            res = getResponse(neighbors)
            predictions.append(res)
        accuracy = getAccuracy(y_test, predictions)
        if(best_acc < accuracy):
            best_acc = accuracy
            best_k = i
            
    print('Accuracy: ', accuracy)
    print('best k :', best_k)

main()


Accuracy:  0.4375
best k : 1
